## Backfill Feature Store

In [1]:
HOPSWORKS_PROJECT_NAME = 'Taxi_Predictor'

In [2]:
#If src not found
import sys
sys.path.insert(0, "/Users/Noah/Taxi_demand_predictor")  # Assuming this is the correct path to project_root

In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')
#e9uU8oCAakG6v52B.wG7RCO0Ha1fC2N3tBXtnSVWujPbNYkrIlqJBIOgq0tupPGyT3RDc6TaaIrzo5H05
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [4]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2023
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
2023-06 file is not available
2023-07 file is not available
2023-08 file is not available
2023-09 file is not available
2023-10 file is not available
2023-11 file is not available
2023-12 file is not available


In [5]:
from src.data import transform_raw_data_into_ts_data
rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'])
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:02<00:00, 96.04it/s] 


In [6]:
import hopsworks

project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/84130


In [7]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [8]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [9]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [10]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/84130/fs/84049/fg/88695


Uploading Dataframe: 100.00% |██████████| Rows 960360/960360 | Elapsed Time: 00:30 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/84130/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x114851090>, None)

## Feature Pipeline

In [47]:
import src.config as config

In [48]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2023-08-05 15:00:00')


In [49]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [50]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-07-08 15:00:00'), to_date_=Timestamp('2022-08-06 15:00:00')
File 2022-07 was already in local storage
File 2022-08 was already in local storage


In [51]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

100%|██████████| 265/265 [00:01<00:00, 207.39it/s]


In [52]:
# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/84130
Connected. Call `.close()` to terminate connection gracefully.


In [53]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 178080/178080 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/84130/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1406df130>, None)

## Model Training Pipeline

In [18]:
%load_ext autoreload
%autoreload 2

In [19]:
import src.config as config

In [20]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/84130
Connected. Call `.close()` to terminate connection gracefully.


In [21]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/84130/fs/84049/fv/time_series_hourly_feature_view/version/1


In [22]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

2023-08-05 11:14:10,949 INFO: USE `taxi_predictor_featurestore`
2023-08-05 11:14:11,820 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_predictor_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '1970-01-01 12:16:40.000' AND `fg0`.`pickup_hour` < TIMESTAMP '2023-08-05 03:14:07.000'


In [23]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
259228,2023-01-01 00:00:00,0,1
84143,2023-01-01 01:00:00,0,1
954869,2023-01-01 02:00:00,0,1
1055677,2023-01-01 03:00:00,0,1
483759,2023-01-01 04:00:00,0,1
...,...,...,...
186153,2023-08-04 23:00:00,5,265
390681,2023-08-05 00:00:00,7,265
370603,2023-08-05 01:00:00,3,265
149480,2023-08-05 02:00:00,12,265


In [24]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [25]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

  0%|          | 0/265 [00:00<?, ?it/s]/Users/Noah/Taxi_demand_predictor/src/data.py:242: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/Users/Noah/Taxi_demand_predictor/src/data.py:242: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/pandas/core/dtype

features_and_target.shape=(41870, 675)


In [26]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29 00:00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,...,1.0,2.0,1.0,1.0,0.0,0.0,0.0,2023-01-29 23:00:00,1,0.0
2,2.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,2.0,1.0,2.0,0.0,1.0,0.0,2023-01-30 22:00:00,1,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,2.0,0.0,0.0,0.0,2023-01-31 21:00:00,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,1.0,0.0,2.0,1.0,1.0,0.0,2023-02-01 20:00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41865,0.0,6.0,5.0,4.0,2.0,0.0,2.0,6.0,7.0,8.0,...,5.0,5.0,1.0,1.0,1.0,4.0,7.0,2023-08-01 06:00:00,265,8.0
41866,0.0,6.0,2.0,3.0,3.0,4.0,4.0,0.0,5.0,7.0,...,11.0,8.0,6.0,3.0,3.0,1.0,0.0,2023-08-02 05:00:00,265,3.0
41867,1.0,2.0,3.0,0.0,2.0,0.0,5.0,0.0,10.0,1.0,...,1.0,5.0,6.0,10.0,2.0,1.0,1.0,2023-08-03 04:00:00,265,4.0
41868,0.0,3.0,4.0,4.0,1.0,13.0,3.0,3.0,4.0,1.0,...,5.0,11.0,9.0,13.0,10.0,7.0,6.0,2023-08-04 03:00:00,265,2.0


In [29]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1))

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2023-07-08 00:00:00')
X_train.shape=(34185, 674)
y_train.shape=(34185,)
X_test.shape=(7685, 674)
y_test.shape=(7685,)


In [30]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [31]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2023-08-05 11:19:09,207] A new study created in memory with name: no-name-808feb7b-6613-4ec8-ac55-b7077236a558
/Users/Noah/Taxi_demand_predictor/src/model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/lightgbm/basic.py:701: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/Users/Noah/Taxi_demand_predictor/src/model.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [32]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 120, 'feature_fraction': 0.40939093498973755, 'bagging_fraction': 0.48972872540965584, 'min_child_samples': 53}


In [33]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/lightgbm/basic.py:701: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



[LightGBM] [Warning] feature_fraction is set=0.40939093498973755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40939093498973755
[LightGBM] [Warning] bagging_fraction is set=0.48972872540965584, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48972872540965584
[LightGBM] [Warning] feature_fraction is set=0.40939093498973755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40939093498973755
[LightGBM] [Warning] bagging_fraction is set=0.48972872540965584, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48972872540965584
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.180979 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171901
[LightGBM] [Info] Number of data points in the train set: 34185, number of used features: 676
[LightGBM] [Info] Start training from score 17.411350


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x13a632d40>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.48972872540965584,
                               feature_fraction=0.40939093498973755,
                               min_child_samples=53, num_leaves=120))])

In [34]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.40939093498973755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.40939093498973755
[LightGBM] [Warning] bagging_fraction is set=0.48972872540965584, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48972872540965584
test_mae=8.2141


/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/lightgbm/basic.py:701: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [35]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['/Users/Noah/Taxi_demand_predictor/models/model.pkl']

In [36]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [37]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(MODELS_DIR / 'model.pkl')

Connected. Call `.close()` to terminate connection gracefully.


Uploading input_example and model_schema:  33%|███▎      | 2/6 [00:27<01:05, 16.33s/it]/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

Model export complete: 100%|██████████| 6/6 [00:34<00:00,  5.81s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/84130/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)

## Inference Pipeline

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2023-08-05 15:00:00')


In [40]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/84130
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-07-08 15:00:00 to 2023-08-05 14:00:00


/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning:

ssl.PROTOCOL_TLS is deprecated



2023-08-05 11:25:45,474 INFO: USE `taxi_predictor_featurestore`
2023-08-05 11:25:45,764 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_predictor_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-07-07 03:00:00.000' AND `fg0`.`pickup_hour` < TIMESTAMP '2023-08-06 02:00:00.000'


/Users/Noah/Taxi_demand_predictor/.venv/lib/python3.10/site-packages/hsfs/engine/python.py:115: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [46]:
import requests

# Define your Hopsworks project ID and model details
project_id = "84130"
model_name = "taxi_demand_predictor_next_hour"
model_version = "1"  # Provide the version number or None to check all versions

# Define the Hopsworks API base URL
api_base_url = "https://c.app.hopsworks.ai:443/p/84130/models/taxi_demand_predictor_next_hour/1"

# Construct the API endpoint URL for the model
if model_version:
    endpoint = f"/project/{project_id}/modelregistries/{project_id}/models/{model_name}/versions/{model_version}"
else:
    endpoint = f"/project/{project_id}/modelregistries/{project_id}/models/{model_name}"

# Make the API request to check if the model exists
response = requests.get(api_base_url + endpoint)

if response.status_code == 200:
    print(f"Model '{model_name}' exists in the project's model registry.")
else:
    print(f"Model '{model_name}' does not exist in the project's model registry.")


Model 'taxi_demand_predictor_next_hour' does not exist in the project's model registry.


In [42]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/84130
Connected. Call `.close()` to terminate connection gracefully.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/84130/modelregistries/84130/models/taxi_demand_predictor_next_hour_2). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":360000,"errorMsg":"No model found for provided name and version."}', error code: 360000, error msg: No model found for provided name and version., user msg: 

In [ ]:
predictions['pickup_hour'] = current_date
predictions

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})
